In [2]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from autocorrect import Speller
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('spam.csv')

In [4]:
data = data[['v1','v2']]

In [5]:
data.rename(columns = {'v1':'labels', 'v2':'email'}, inplace = True)

## Let's look at the distribution of words for each email type.

### Spam emails

In [6]:
spam_words = ' '.join(list(data[data['labels'] == 'spam']['email']))
#This will allow us to visualize the most common words that happear in a spam email.
spam_wc = WordCloud(width=500, height = 500).generate(spam_words)
plt.figure(figsize = (10,8))
plt.imshow(spam_wc)
plt.show()

### Ham emails

In [7]:
spam_words = ' '.join(list(data[data['labels'] == 'ham']['email']))
#This will allow us to visualize the most common words that happear in a spam email.
spam_wc = WordCloud(width=500, height = 500).generate(spam_words)
plt.figure(figsize = (10,8))
plt.imshow(spam_wc)
plt.show()

In [8]:
data.labels.value_counts()

ham     4825
spam     747
Name: labels, dtype: int64

# Preprocessing the data

- Remove characters that are not letters
- apply .lower() to every word
- tokenization
- remove the stop words
- stemming 
- Correct the spelling

Removing non common characters. 

In [9]:
data.email = data.email.apply(lambda x: re.sub('[^A-Za-z]',' ', x))

Apply .lower()

In [10]:
data.email = data.email.str.lower()

Tokenization

In [11]:
data.email = data.email.apply(lambda x: word_tokenize(x))

Removing stop words, correct spelling and stemming

In [12]:
spell_correcter = Speller()
stemmer  = PorterStemmer()
list_of_emails = []
for row in tqdm(data.email):
    lst = []
    for word in row:
        #remove the words that are stopwords
        if word not in stopwords.words('english'):
            #corrects the spelling
            corrected_word = spell_correcter(word)
            #stems each word
            word_stemm = stemmer.stem(corrected_word)
            #makes a new sentence without the stop words, with the word stemmed and without spelling errors
            lst.append(word_stemm)
    #appends to the big list
    list_of_emails.append(lst)

100%|██████████| 5572/5572 [06:26<00:00, 14.40it/s]


In [13]:
new_lst = []
for i in list_of_emails:
    sentence = ' '.join(i)
    new_lst.append(sentence)


Splitting the data

In [14]:
x_train, x_test, y_train, y_test = train_test_split(new_lst, data.labels, test_size=0.25, random_state=42)

Vectorize the emails

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
counter = CountVectorizer()

In [16]:
x_train = counter.fit_transform(x_train).toarray()
x_test = counter.transform(x_test).toarray()


In [17]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [18]:
model.fit(x_train, y_train)

GaussianNB()

In [19]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_test, model.predict(x_test)))
print('Accuracy:\n', accuracy_score(y_test, model.predict(x_test)))

              precision    recall  f1-score   support

         ham       0.98      0.85      0.91      1202
        spam       0.48      0.86      0.62       191

    accuracy                           0.85      1393
   macro avg       0.73      0.86      0.76      1393
weighted avg       0.91      0.85      0.87      1393

Accuracy:
 0.8521177315147165


Since our problem is a binary problem, we can use a logistic regression model.

In [20]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train, y_train)
print(classification_report(y_test, model.predict(x_test)))
print('Accuracy:\n', accuracy_score(y_test, model.predict(x_test)))

              precision    recall  f1-score   support

         ham       0.97      1.00      0.99      1202
        spam       0.99      0.82      0.90       191

    accuracy                           0.97      1393
   macro avg       0.98      0.91      0.94      1393
weighted avg       0.98      0.97      0.97      1393

Accuracy:
 0.9748743718592965


This approach leads to a much better accuracy